## Lagrange

In [ ]:
def lagrange_basis(x, x_data_l, k):
    """ Calcula la base de Lagrange L_k(x). """
    basis = 1
    for i in range(len(x_data_l)):
        if i != k:
            basis *= (x - x_data_l[i]) / (x_data_l[k] - x_data_l[i])
    return basis

def lagrange_interpolation(x_data_l, y_data_l, x):
    """ Calcula el polinomio de interpolación de Lagrange en x. """
    n = len(x_data_l)
    result = 0
    for k in range(n):
        result += y_data_l[k] * lagrange_basis(x, x_data_l, k)
    return result

# Cargar los datos
data = np.loadtxt("Datos_Taller_2.txt")
x_data, y_data = data[:, 0], data[:, 1]

# Puntos para la evaluación del polinomio
x_vals_l = np.linspace(0, 6, 100)
y_vals_l = [lagrange_interpolation(x_data, y_data, x) for x in x_vals_l]

# Graficar
plt.scatter(x_data, y_data, color='red', label='Datos experimentales')
plt.plot(x_vals_l, y_vals_l, label='Polinomio de interpolación de Lagrange', color='blue')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.title('Interpolación de Lagrange')
plt.grid()
plt.show()